Python version 3.9.7

# Imports

Here we load all the imports we need to run the notebook

In [1]:
import pandas as pd
from py2neo import Graph
from py2neo.bulk import create_nodes
import os
import sys
import requests
import pandas as pd
import numpy as np
from IPython.display import display
from time import sleep
from tqdm import tqdm
import json
import re
import time
from py2neo import Graph, Node, Relationship
from py2neo.matching import *
from py2neo import Graph
from py2neo.bulk import create_relationships
from itertools import islice
from IPython.display import display

In [2]:
start = time.time()

# Functions

This function will help us later.
If we have a dataframe column that consists of lists.
Imagine a row that has an id value in the fist column and in the second column it has lists
Lets say id would be 1 and the list would be [cat,dog]

After the unnest function we would get 2 rows that would be

Row1 : 1 [cat]

Row2 : 1 [dog]

In [3]:
def unnest(df, col, reset_index=False):
    import pandas as pd
    col_flat = pd.DataFrame([[i, x] 
                       for i, y in df[col].apply(list).iteritems() 
                           for x in y], columns=['I', col])
    col_flat = col_flat.set_index('I')
    df = df.drop(col, 1)
    df = df.merge(col_flat, left_index=True, right_index=True)
    if reset_index:
        df = df.reset_index(drop=True)
    return df

# Load File and Graph

Here we create the graph, setup the options that our dataframe will have and load the dataset from the csv file

Notice that we drop duplicates based on the id_str column. We do that because each tweet is supposed to have a unique id, either it is tweet or a retweet. So, we assume that the duplicate is a fault and we drop them

In [4]:
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows
pd.set_option('display.float_format', '{:.0f}'.format)
pd.options.display.float_format = '{:.0f}'.format
tweets = pd.read_csv(r'C:\Users\xaris\Desktop\webmining\TwitterData.csv')
tweets = tweets.drop_duplicates(subset=['id_str']).reset_index(drop=True)
tweets = tweets.dropna(subset = ['id_str'])
g = Graph(password='0000')
graph = Graph(password='0000')

# Tweet Node

We create 2 dataframes: one for the normal tweets (original_tweets) and one for the retweeted tweets (retweeted_tweets)

In [5]:
original_tweets = tweets[tweets['retweeted_status.id_str'].isnull()]
retweeted_tweets = tweets.dropna(subset = ['retweeted_status.id_str'])

We now choose what columns we will use

In [6]:
original_tweets = original_tweets[['id_str',
                                   'user.id_str',
                                   'created_at',
                                   'source',
                                   'text',
                                   'entities.hashtags',
                                   'entities.urls',
                                   'entities.user_mentions',
                                   'user.screen_name',
                                   'user.created_at',
                                   'user.followers_count']]
original_tweets = original_tweets.reset_index(drop = True)

In [7]:
retweeted_tweets = retweeted_tweets[['id_str',
                                     'user.id_str',
                                     'retweeted_status.id_str',
                                     'retweeted_status.created_at',
                                     'retweeted_status.source',
                                     'retweeted_status.text',
                                     'retweeted_status.entities.hashtags',
                                     'entities.user_mentions',
                                     'retweeted_status.entities.urls',
                                     'retweeted_status.entities.user_mentions',
                                     'retweeted_status.user.created_at',
                                     'retweeted_status.user.followers_count']]
retweeted_tweets = retweeted_tweets.reset_index(drop = True)

Below we get the original author and his/her username for the retweets.
Notice that we look up at the 'entities.user_mentions' column.
In a retweet you can find the author of the original tweet as the first person mentioned in the 'entities.user_mentions' of the retweet.

In [8]:
original_author = {}
for t in retweeted_tweets.index:
    user_mentioned_json = retweeted_tweets["entities.user_mentions"][t]
    user_mentioned_loaded= json.loads(user_mentioned_json)
    i=0
    for screen_names in range(len(user_mentioned_loaded)):
        if i==0 :
            user_name_mentioned = user_mentioned_loaded[screen_names]['screen_name'] 
            user_mentioned_id = user_mentioned_loaded[screen_names]['id_str']
            original_author.update({retweeted_tweets['id_str'][t]:(user_mentioned_id,user_name_mentioned)})
            break
#we break as soon as we find the first one cause thats the only one we need

We rename and rearrange the retweeted_tweets columns to match the original_tweets so we can merge them later

In [ ]:
Original_authors = pd.DataFrame(original_author.values(), columns=['user.id_str', 'user.screen_name'])
retweeted_tweets.drop('id_str', axis=1, inplace=True)
retweeted_tweets.drop('entities.user_mentions', axis=1, inplace=True)
retweeted_tweets.drop('user.id_str', axis=1, inplace=True)
retweeted_tweets = pd.concat([retweeted_tweets, Original_authors], axis=1,join='inner')
retweeted_tweets = retweeted_tweets.rename(columns={'retweeted_status.id_str': 'id_str',
                                                    'retweeted_status.created_at':'created_at',
                                                    'retweeted_status.source':'source',
                                                    'retweeted_status.text':'text',
                                                    'retweeted_status.entities.hashtags':'entities.hashtags',
                                                    'retweeted_status.entities.urls':'entities.urls',
                                                    'retweeted_status.entities.user_mentions':'entities.user_mentions',
                                                    'retweeted_status.user.created_at':'user.created_at',
                                                    'retweeted_status.user.followers_count': 'user.followers_count'})

In [10]:
retweeted_tweets = retweeted_tweets[['id_str',
                                     'user.id_str',
                                     'created_at',
                                     'source',
                                     'text', 
                                     'entities.hashtags',
                                     'entities.urls',
                                     'entities.user_mentions',
                                     'user.screen_name',
                                     'user.created_at',
                                     'user.followers_count']]

We merge the 2 datasets into one and drop the duplicates that might exist. This could be the case cause the dataset could have an original tweet and its retweets but we only need one to create the node.

Counts dataframe will be used to create usl and hashtagnodes

In [11]:
original_final = pd.concat([original_tweets,retweeted_tweets],ignore_index=False)
original_final.reset_index(drop=True,inplace=True)
counts = original_final
original_final = original_final.drop_duplicates(subset=['id_str'],keep = 'first').reset_index(drop=True)

We use the py2neo.bulk to load our data fast. Our dataset is relatively small so just seeting a batch size is fine. If your dataset is larger you should set a lower batch size(1/5 of your data would be a good start) and then loop the batches.

In [12]:
#to use bulk we need to send a list of dicts and thats what to_dict('records') does.
unique_tweets = original_final.to_dict('records')
stream = iter(unique_tweets)
batch_size = 50000
batch = islice(stream, batch_size)
create_nodes(graph.auto(), batch, labels={"Tweet"})

# Hashtag Node

We create the Hashtage Nodes (Same logic will be applied in the Url Nodes too). We reform both counts dataframe and original_final datframe from json format to a list format.

We could just do the reform on only the original_final. But there is a error in the dataframe that makes us lose hashtags and urls. Before dropping the duplicates in the original_final we find out that we have similar tweet ids with differnt hashtags/urls inside them. So we created the counts so we dont lose any valuable info in the form or hashtags/urls. You can see this as a work around to fix the error our data has.

In [13]:
for t in counts.index:
    hash_in_text = counts["entities.hashtags"][t]
    hash_list=[]
    hashtags=json.loads(hash_in_text)
    for k in range(len(hashtags)):
        hash_list.append(hashtags[k]['text'].lower())
    counts['entities.hashtags'][t] = hash_list
for t in original_final.index:
    hash_in_text = original_final["entities.hashtags"][t]
    hash_list=[]
    hashtags=json.loads(hash_in_text)
    for k in range(len(hashtags)):
        hash_list.append(hashtags[k]['text'].lower())
    original_final['entities.hashtags'][t] = hash_list

C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/2892804214.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts['entities.hashtags'][t] = hash_list
C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/2892804214.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_final['entities.hashtags'][t] = hash_list


Same as Tweet nodes we create the hashtag nodes after we use the unnest function.

In [14]:
hashtag_node = counts[['entities.hashtags']]
hashtag_node = hashtag_node[hashtag_node['entities.hashtags'].map(lambda d: len(d)) > 0]
hashtag_node = unnest(hashtag_node, 'entities.hashtags')
hashtag_node = hashtag_node.reset_index(drop=True)
unique_hashtags = hashtag_node.drop_duplicates(subset=['entities.hashtags']).reset_index(drop=True)
hashtag_dict = unique_hashtags.to_dict('records')
stream = iter(hashtag_dict)
batch_size = 50000
batch = islice(stream, batch_size)
create_nodes(graph.auto(), batch, labels={"Hashtag"})

C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/2324800661.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(col, 1)


# Url Node

In [15]:
for t in counts.index:
    url_in_text = counts["entities.urls"][t]
    url_list=[]
    urls=json.loads(url_in_text)
    for k in range(len(urls)):
        url_list.append(urls[k]["url"])
    counts['entities.urls'][t] = url_list
for t in original_final.index:
    url_in_text = original_final["entities.urls"][t]
    url_list=[]
    urls=json.loads(url_in_text)
    for k in range(len(urls)):
        url_list.append(urls[k]["url"])
    original_final['entities.urls'][t] = url_list

C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/1921096117.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts['entities.urls'][t] = url_list
C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/1921096117.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_final['entities.urls'][t] = url_list


In [16]:
url_node = counts[['entities.urls']]
url_node = url_node[url_node['entities.urls'].map(lambda d: len(d)) > 0]
url_node = unnest(url_node, 'entities.urls')
url_node = url_node.reset_index(drop=True)
unique_urls = url_node.drop_duplicates(subset=['entities.urls']).reset_index(drop=True)
url_dict = unique_urls.to_dict('records')
stream = iter(url_dict)
batch_size = 50000
batch = islice(stream, batch_size)
create_nodes(graph.auto(), batch, labels={"Url"})

C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/2324800661.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(col, 1)


# User Node

To get all the users we need to get the authors of original tweets, the users that retweeted tweets and all the mentioned users.

In [17]:
mentioned_users_temp = {}
for t in tweets.index:
    user_mentioned_json = tweets["entities.user_mentions"][t]
    user_mentioned_loaded= json.loads(user_mentioned_json)
    for screen_names in range(len(user_mentioned_loaded)):    
        user_name_mentioned = user_mentioned_loaded[screen_names]['screen_name'] 
        user_mentioned_id = user_mentioned_loaded[screen_names]['id_str'] 
        temp_dict = {user_mentioned_id:user_name_mentioned}
        mentioned_users_temp.update(temp_dict)
#To make the loop faster  we use a dictionary and then we turn the dictionary in a dataframe.
mentioned_users = pd.DataFrame(list(mentioned_users_temp.items()),columns = ['user.id_str','user.screen_name']) 

In [18]:
#we merge all those 3 dataframes and then we drop duplicates.
#The keep='first' is a crusial part so that we can keep the user with the most info (eg a mentioned use would not have a followers count)
users = tweets[['user.id_str','user.screen_name','user.created_at','user.followers_count']]#tweets[['user.id_str','user.screen_name','user.created_at','user.followers_count']]
users_rt = retweeted_tweets[['user.id_str','user.screen_name','user.created_at','user.followers_count']]
original_tweets[['user.id_str','user.screen_name','user.created_at','user.followers_count']]
final_users = pd.concat([users,users_rt, mentioned_users],ignore_index=False)
final_users = final_users.drop_duplicates(subset=['user.id_str'],keep = 'first').reset_index(drop=True)
unique_users = final_users.to_dict('records')

In [21]:
stream = iter(unique_users)
batch_size = 50000
batch = islice(stream, batch_size)
create_nodes(graph.auto(), batch, labels={"User"})

# Relationship Tweet

To create the Tweet relationship between the user and the tweet (and all relationships in general) as fast as possible we need to create a list of tuples that has the form of (id of the first node,{relationship attributes},id of the other node). In this way the system doesn't search all the nodes to find the one that matches the criteria. On the other hand, this way works like a hash table (the node ids are unique) so the cost is considerably lower.

In [57]:
matcher=NodeMatcher(g)
nodes=matcher.match("Tweet")
master_dict_node = {}
master_dict_rel = {}
tweetid_nodeid = {}
#parseing the nodes
for node in nodes:
    #gets the node id
    nodeid = node.identity
#for each node we get the respective attributes we are gonna need to create the relationship
    tweet_id = (list(node.values())[4])
    tweetid_nodeid.update({tweet_id:nodeid})
    userid = (list(node.values())[0])
    date = (list(node.values())[5])
    source = (list(node.values())[6])
#we create 2 dictionaries(we try to be fast and dictionaries and exceptionally fast in python)
#master_dict_rel is a dictionary that contains the attributes of each relationship we need to create
#the key is the id of the tweet node and the values are the date and source of this node
    master_dict_rel.update({nodeid:(date,source)})
#master_dict_node is dictionary that has a key in the form of the id of the node and value in the form of the id of the user created the tweet
    master_dict_node.update({nodeid:userid})

In [23]:
matcher=NodeMatcher(g)
nodes=matcher.match("User")
master_user_dict = {}
#here we create master_user_dict while parsing the nodes
#master_user_dict is a dictionary that has a key in the form of the id of the user and value in the form of the id of the node that represents the user
for node in nodes:
    nodeid = node.identity
    userid = (list(node.values())[0])
    master_user_dict.update({userid:nodeid})

In [24]:
test_list = []
#we parse the master_dict_node dictionary and get the values. Each value we take we take we use to to access the master_user_dict and get the node id.
#we also get the source and the date and create a small dictionary with them.
for key in master_dict_node.keys():
    a = master_dict_node.get(key)
    b = master_user_dict.get(a)
    c = master_dict_rel[key][0]
    d = master_dict_rel[key][1]
    temp_dict = {'Created at':c,'Source':d}
    temp_tuple = (key,temp_dict,b)
    test_list.append(temp_tuple)

In [25]:
create_relationships(g.auto(), test_list, "Tweeted")

# Relationship Retweeted

The logic is like the previous relationship. 
The main difference is that we get only the tweets that were retweeted.

In [26]:
rel_rt = tweets.dropna(subset = ['retweeted_status.id_str'])
rel_rt = rel_rt[['id_str',
                 'user.id_str',
                 'retweeted_status.id_str',
                 'retweeted_status.created_at',
                 'retweeted_status.source',
                 'retweeted_status.text',
                 'retweeted_status.entities.hashtags',
                 'entities.user_mentions',
                 'retweeted_status.entities.urls',
                 'retweeted_status.entities.user_mentions',
                 'retweeted_status.user.created_at',
                 'retweeted_status.user.followers_count']]
rel_rt = rel_rt.reset_index(drop = True)
Original_authors_rel = pd.DataFrame(original_author.values(), columns=['Author', 'Authot_name'])
rel_rt = pd.concat([rel_rt, Original_authors_rel], axis=1,join='inner')

In [27]:
#The master_rt_dict is a dictionary that has each retweet id as a key and as value the id of the original tweet and the user that retweeted the tweet.
master_rt_dict = {}
rt_rl_dict = {}
for t in range(len(rel_rt)):
    master_rt_dict.update({rel_rt['id_str'][t]:(rel_rt['retweeted_status.id_str'][t],rel_rt['user.id_str'][t])})
    rt_rl_dict.update({rel_rt['id_str'][t]:(rel_rt['retweeted_status.user.created_at'][t],rel_rt['retweeted_status.source'][t])})

In [28]:
#same as tweeted relationship we create the retweeted relationship
test_list = []
for key in master_rt_dict.keys():
    id_str_og_tweet = master_rt_dict[key][0]
    rt_user_id_str = master_rt_dict[key][1]
    node_user_id = master_user_dict[rt_user_id_str]
    node_tweet_id = tweetid_nodeid[id_str_og_tweet]
    created = rt_rl_dict[key][0]
    source = rt_rl_dict[key][1]
    temp_dict = {'Created at':created,'Source':source}
    temp_tuple = (node_tweet_id,temp_dict,node_user_id)
    test_list.append(temp_tuple)

In [52]:
print(tweetid_nodeid[1.4947808842512507e+18])

168051


In [29]:
create_relationships(g.auto(), test_list, "Retweeted")

# Relationship Has Hashtag

We find all the tweets that have a hashtag and create a dictionary with the id of the user that tweeted the tweet and a list with all the hashtags that the tweet contained

We also create a dictionary(hashtag_dict_node) that has the form of the hashtag name as the key and the node id that represents that hashtag as a value

In [30]:
hashtag_dict = {}
for t in range(len(original_final)):
    if len(original_final['entities.hashtags'][t])!=0:
        hashtag_dict.update({original_final['id_str'][t]:original_final['entities.hashtags'][t]})
matcher=NodeMatcher(g)
nodes=matcher.match("Hashtag")
hashtag_dict_node = {}
for node in nodes:
    nodeid = node.identity
    hashtagid = (list(node.values())[0])
    hashtag_dict_node.update({hashtagid:nodeid})

In [31]:
test_list = []
temp_dict = {}
for key in hashtag_dict.keys():
    a = len(hashtag_dict[key])
#for each value in hashtag_dict_node so for each hashtag we get the node that this hashtag is located.
#after that we with the tweet id the node that this tweet represents through tweetid_nodeid
#and like above we create a a tuple and pass it to a list so we can create the relationship
    for i in range(a):
        hashid = hashtag_dict_node[hashtag_dict[key][i]]
        tweetid = tweetid_nodeid[key]
        temp_tuple = (hashid,temp_dict,tweetid)
        test_list.append(temp_tuple)

In [32]:
create_relationships(g.auto(), test_list, "Has Hashtag")

# Relationship Has Url

Same technique as avove in the has hashtag relationship

In [33]:
url_dict = {}
for t in range(len(original_final)):
    if len(original_final['entities.urls'][t])!=0:
        url_dict.update({original_final['id_str'][t]:original_final['entities.urls'][t]})
matcher=NodeMatcher(g)
nodes=matcher.match("Url")
url_dict_node = {}
for node in nodes:
    nodeid = node.identity
    urlid = (list(node.values())[0])
    url_dict_node.update({urlid:nodeid})

In [34]:
test_list = []
temp_dict = {}
for key in url_dict.keys():
    a = len(url_dict[key])
    for i in range(a):
        urlid = url_dict_node[url_dict[key][i]]
        tweetid = tweetid_nodeid[key]
        temp_tuple = (urlid,temp_dict,tweetid)
        test_list.append(temp_tuple)

In [35]:
create_relationships(g.auto(), test_list, "Has Url")

# Relationship Used Hashtag 
## (for normal tweets)


For this  relationship we need to break our data into 2 parts. One for the normaly tweeted tweets and one for the retweeted ones. The same logic applies to both of them: the only thing that changes is the dataframe that it is applied on.

In [36]:
hashtag_dict = {}
userid_dict = {}
test_list = []
temp_dict = {}
for t in range(len(original_final)):
    if len(original_final['entities.hashtags'][t])!=0:
#as we have done countless times before we create dictionaries that have as a key the id of the tweet and as values the user id on the first one and the hashtags as a list in the second one
        hashtag_dict.update({original_final['id_str'][t]:original_final['user.id_str'][t]})
        userid_dict.update({original_final['id_str'][t]:original_final['entities.hashtags'][t]})
#same logic as has hashtag relationship
for key in hashtag_dict:
    user_node_id = master_user_dict[hashtag_dict[key]]
    for i in range(len(userid_dict[key])):
        b = userid_dict[key][i]
        hash_node_id = hashtag_dict_node[b]
        temp_tuple = (user_node_id,temp_dict,hash_node_id)
        test_list.append(temp_tuple)

In [37]:
create_relationships(g.auto(), test_list, "Used Hashtag")

# Relationship Used Hashtag 
## (for retweets)

In [39]:
for t in rel_rt.index:
    hash_in_text = rel_rt['retweeted_status.entities.hashtags'][t]
    hash_list=[]
    hashtags=json.loads(hash_in_text)
    for k in range(len(hashtags)):
        hash_list.append(hashtags[k]['text'].lower())
    rel_rt['retweeted_status.entities.hashtags'][t] = hash_list

C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/2282643530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_rt['retweeted_status.entities.hashtags'][t] = hash_list


In [40]:
hashtag_dict = {}
userid_dict = {}
test_list = []
temp_dict = {}
for t in range(len(rel_rt)):
    if len(rel_rt['retweeted_status.entities.hashtags'][t])!=0:
        hashtag_dict.update({rel_rt['id_str'][t]:rel_rt['user.id_str'][t]})
        userid_dict.update({rel_rt['id_str'][t]:rel_rt['retweeted_status.entities.hashtags'][t]})
for key in hashtag_dict:
    user_node_id = master_user_dict[hashtag_dict[key]]
    for i in range(len(userid_dict[key])):
        b = userid_dict[key][i]
        hash_node_id = hashtag_dict_node[b]
        temp_tuple = (user_node_id,temp_dict,hash_node_id)
        test_list.append(temp_tuple)

In [41]:
create_relationships(g.auto(), test_list, "Used Hashtag")

# Relationship Used Url
## (for Normal Tweets)

Like we did for the Has Hashtag relationship for both tweets and retweets, we do the same for the url part

In [42]:
url_dict = {}
userid_dict = {}
test_list = []
temp_dict = {}
for t in range(len(original_final)):
    if len(original_final['entities.urls'][t])!=0:
        url_dict.update({original_final['id_str'][t]:original_final['user.id_str'][t]})
        userid_dict.update({original_final['id_str'][t]:original_final['entities.urls'][t]})
for key in url_dict:
    user_node_id = master_user_dict[url_dict[key]]
    for i in range(len(userid_dict[key])):
        b = userid_dict[key][i]
        url_node_id = url_dict_node[b]
        temp_tuple = (user_node_id,temp_dict,hash_node_id)
        test_list.append(temp_tuple)

In [43]:
create_relationships(g.auto(), test_list, "Used Url")

# Relationship Used Url
## (For Normal Tweets)

In [44]:
for t in rel_rt.index:
    url_in_text = rel_rt["retweeted_status.entities.urls"][t]
    url_list=[]
    urls=json.loads(url_in_text)
    for k in range(len(urls)):
        url_list.append(urls[k]["url"])
    rel_rt['retweeted_status.entities.urls'][t] = url_list

C:\Users\xaris\AppData\Local\Temp/ipykernel_22780/648739987.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_rt['retweeted_status.entities.urls'][t] = url_list


In [45]:
userid_dict = {}
url_dict = {}
test_list = []
temp_dict = {}
for t in range(len(rel_rt)):
    if len(rel_rt['retweeted_status.entities.urls'][t])!=0:
        url_dict.update({rel_rt['id_str'][t]:rel_rt['user.id_str'][t]})
        userid_dict.update({rel_rt['id_str'][t]:rel_rt['retweeted_status.entities.urls'][t]})
for key in url_dict:
    user_node_id = master_user_dict[url_dict[key]]
    for i in range(len(userid_dict[key])):
        b = userid_dict[key][i]
        url_node_id = url_dict_node[b]
        temp_tuple = (user_node_id,temp_dict,hash_node_id)
        test_list.append(temp_tuple)

In [46]:
create_relationships(g.auto(), test_list, "Used Url")

# Mentioned

Now for the last part we need to find and connect who mentioned who.

Just like the has hashtag relationship we have 2 loops one inside the other.

In [47]:
mentioned_users
master_user_dict
mentions = tweets[['user.id_str','entities.user_mentions']]

In [48]:
test_list = []
temp_dict = {}
for i in mentions.index:
#for each user we refer to the dictionaries we have created before.
# Reform the json as we did before
    current_user = mentions['user.id_str'][i]
    a = master_user_dict[current_user]
    user_mentioned_json = mentions['entities.user_mentions'][i]
    user_mentioned_loaded= json.loads(user_mentioned_json)
    for screen_names in range(len(user_mentioned_loaded)):
#for each user we refer to master_user_dict to get the node id 
#this way we get both users node id so we create the tuple
        user_mentioned_id = user_mentioned_loaded[screen_names]['id_str']
        b = master_user_dict[user_mentioned_id]
        temp_tuple = (a,temp_dict,b)
        test_list.append(temp_tuple)
        
        

In [49]:
create_relationships(g.auto(), test_list, "Mention")

In [50]:
end = time.time()
print(end - start)

58.67172050476074


The shown time is from a pc that uses an i74790k(2014 cpu) and 16bg of ddr3 1600hz memory